In [ ]:
# Application of FL task
from MLModel import *
from FLModel import *
from utils import *

import torch
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

In [ ]:
data_set = 'news_popular'

def load_credit2(feature, kid):
    non_iid = []
    test = None
    for c in range(1, 5):
        X_test = np.load("model/"+data_set+'/'+feature+"/c"+str(kid)+"/test_x"+str(c)+".npy")
        y_test = np.load("model/"+data_set+'/'+feature+"/c"+str(kid)+"/test_y"+str(c)+".npy")
        X_train = np.load("model/"+data_set+'/'+feature+"/c"+str(kid)+"/train_x"+str(c)+".npy")
        y_train = np.load("model/"+data_set+'/'+feature+"/c"+str(kid)+"/train_y"+str(c)+".npy")
        if test:
            test[0] = np.concatenate((test[0], X_test), axis=0)
            test[1] = np.concatenate((test[1], y_test), axis=0)
        else:
            test = [X_test, y_test]
        non_iid.append((X_train, y_train))
    non_iid.append((test[0], test[1]))
    return non_iid

feature = 'n_tokens_title'
client_num = 4

## LR model

In [ ]:
import warnings
warnings.filterwarnings("ignore")

for eps in [0.25, 0.5, 1, 2, 4, 8, 1e12]: # 
    print("==========")
    noise = compute_noise(1, 0.05, eps, 500, 1e-4, 1e-5)
    avg_acc = []
    
    for kid in range(1, 6):
        d = load_credit2(feature, kid)
        for _ in range(5):
            lr = 0.15
            fl_param = {
                'output_size': 5,
                'client_num': client_num,
                'model': LogisticRegression,
                'data': d,
                'lr': lr,
                'E': 100,
                'C': 1,
                'eps': eps,
                'delta': 1e-4,
                'noise': noise,
                'q': 0.05,
                'clip': 4,
                'tot_T': 5,
                'batch_size': 32,
                'pth': "model/"+data_set+'/'+feature+"/c"+str(kid)+"/"+str(_+1)+"/",
                'device': device
            }

            fl_entity = FLServer(fl_param).to(device)
            acc = []
            for e in range(5):
                acc += [fl_entity.global_update(e+1)]
            avg_acc.append(acc[-1])
    print("eps=", eps, "| avg acc =", np.mean(avg_acc))

## MLP model

In [ ]:
import warnings
warnings.filterwarnings("ignore")

for eps in [0.25, 0.5, 1, 2, 4, 8, 1e12]: # 
    noise = compute_noise(1, 0.05, eps, 500, 1e-4, 1e-5)
    avg_acc = []
    
    for kid in range(1, 6):
        d = load_credit2(feature, kid)
        
        for _ in range(5):
            lr = 0.15
            fl_param = {
                'output_size': 5,
                'client_num': client_num,
                'model': LightMLP2,
                'data': d,
                'lr': lr,
                'E': 100,
                'C': 1,
                'eps': eps,
                'delta': 1e-4,
                'noise': noise,
                'q': 0.05,
                'clip': 4,
                'tot_T': 5,
                'batch_size': 32,
                'pth': "model/"+data_set+'/'+feature+"/c"+str(kid)+"/"+str(_+1)+"/",
                'device': device
            }

            fl_entity = FLServer(fl_param).to(device)
            acc = []
            for e in range(5):
                acc += [fl_entity.global_update(e+1)]
            avg_acc.append(acc[-1])
    print("eps=", eps, "| avg acc =", np.mean(avg_acc))